In [1]:
import pandas as pd
import datetime

In [2]:
df_messdaten = pd.read_csv("AlleMessdaten2000_2024.csv")
df_wetterdaten = pd.read_csv("Wetterdaten.csv")
df_ferien = pd.read_csv("ferientage_2000_2025.csv")

df_feiertage = pd.read_excel("DeutscheFeiertage.xlsx", 
                             dtype={"Feiertag": "category", 
                                    "Datum": "object", 
                                    "Bundesland": "category"})

df_feiertage["Datum"] = pd.to_datetime(df_feiertage["Datum"]).dt.date

Einige Datensätze hatten keine Zuordnung zu einem Bundesland

array(['Frechen', 'Köln', 'Monheim', 'Bergisch Gladbach', 'Berlin',
       'Bonn', 'Leverkusen', 'Langenfeld', 'Düsseldorf', nan],
      dtype=object)

In [4]:
df_messdaten.loc[
    (df_messdaten["federal_state"] == "Sonstige") & (df_messdaten["city"].isin(['Frechen', 'Köln', 'Monheim', 'Bergisch Gladbach', 'Bonn', 'Leverkusen', 'Langenfeld', 'Düsseldorf'])), 
    "federal_state"
    ] = "Nordrhein-Westfalen"

df_messdaten.loc[
    (df_messdaten["federal_state"] == "Sonstige") & (df_messdaten["city"] == "Berlin"), 
    "federal_state"
    ] = "Berlin"

# Es gibt eine Verbrauchsstelle ohne Bundesland, Ort oder PLZ
# Annahme: NRW
df_messdaten.loc[
    df_messdaten["federal_state"].isna(), 
    "federal_state"
    ] = "Nordrhein-Westfalen"

# Es gibt eine Verbrauchsstelle ohne Bundesland und Ort, aber mit PLZ
df_messdaten.loc[df_messdaten["federal_state"] == "Sonstige", "federal_state"] = "Nordrhein-Westfalen"

In [5]:
# Prüfung, ob Wetterdaten von mehreren Stationen im gleichen Bundesland pro Tag vorliegen
# Wie viele Duplikate gibt es pro (date, state) in den Wetterdaten?
dups = df_wetterdaten.groupby(['date', 'state']).size().reset_index(name='count')
dups = dups[dups['count'] > 1]
print(dups)

                             date            state  count
1       2000-01-01 00:00:00+00:00           Bayern      7
2       2000-01-01 00:00:00+00:00           Berlin      4
3       2000-01-01 00:00:00+00:00      Brandenburg      9
4       2000-01-01 00:00:00+00:00           Bremen      9
5       2000-01-01 00:00:00+00:00          Hamburg      9
...                           ...              ...    ...
117804  2024-12-31 00:00:00+00:00  Rheinland-Pfalz      5
117805  2024-12-31 00:00:00+00:00         Saarland      9
117806  2024-12-31 00:00:00+00:00          Sachsen      5
117807  2024-12-31 00:00:00+00:00   Sachsen-Anhalt      5
117808  2024-12-31 00:00:00+00:00        Thüringen      8

[117371 rows x 3 columns]


Zusammenführen der Datensätze

In [19]:
# Sicherstellen, dass die Datumsformate übereinstimmen
df_messdaten['Zeitstempel'] = pd.to_datetime(df_messdaten['Zeitstempel']).dt.date
df_wetterdaten['date'] = pd.to_datetime(df_wetterdaten['date']).dt.date

agg_dict = {
    'station_id': 'first',
    'resolution': 'first',
    'dataset': 'first',
    'parameter': 'first',
    'value': 'mean',        # numerisch
    'quality': 'min',
    'name': 'first'
}

# Wetterdaten vorher pivotisieren, da sonst Duplikate nach dem Join im DataFrame auftreten
df_wetter_pivot = df_wetterdaten.pivot_table(
    index=['date', 'state'],
    columns='parameter',
    values='value',
    aggfunc='mean'
).reset_index()

# Klimadaten mergen anhand Datum und Bundesland
df = pd.merge(
    df_messdaten,
    df_wetter_pivot,
    left_on=['Zeitstempel', 'federal_state'],
    right_on=['date', 'state'],
    how='left'
)

# doppelte Zeit- und Ortsangaben entfernen
df.drop(columns=['date', 'state'], inplace=True)

# nur die Messwerte beibehalten, die auch Klimainformationen haben
df = df.loc[df["temperature_air_mean_2m"].isna() == False]

# Wochentag
df["Wochentag"] = pd.to_datetime(df["Zeitstempel"]).apply(lambda x: datetime.date.isoweekday(x))

# in Bezeichnung umschlüsseln
dict_weekday = {1: "Montag",
                2: "Dienstag",
                3: "Mittwoch",
                4: "Donnerstag",
                5: "Freitag",
                6: "Samstag",
                7: "Sonntag"}

df["Wochentag_Name"] = df["Wochentag"].apply(lambda x: dict_weekday.get(x))

# Feiertage mergen
df = pd.merge(
    df,
    df_feiertage,
    left_on=['Zeitstempel', 'federal_state'],
    right_on=['Datum', 'Bundesland'],
    how='left'
)

df.drop(columns=['Datum', 'Bundesland'], inplace=True)

# selbes Datumformat zum mergen, hier gab es vorher Probleme ohne Konvertierung des Datums
df["Zeitstempel"] = pd.to_datetime(df["Zeitstempel"])
df_ferien["Datum"] = pd.to_datetime(df_ferien["Datum"])

# Ferien mergen
df = pd.merge(
    df,
    df_ferien,
    left_on=["Zeitstempel", "federal_state"],
    right_on=["Datum", "Bundesland"],
    how="left"
)

df.drop(columns=["Datum", "Bundesland"], inplace=True)

In [24]:
df.to_csv("EnrichedSensorData.csv")